## Purpose
Transform raw repository metrics into actionable business intelligence:
- **Engineered Features**: Ratios, indices, and composite scores
- **Derived Tables**: Aggregated views by language, segment, and readiness tier
- **Export**: CSV files for dashboard integration

In [1]:
# Import required libraries
from pathlib import Path
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

print("✅ Libraries loaded successfully!")

✅ Libraries loaded successfully!


## Load Base Dataset
Read the main combined dataset with all raw repository metrics.

In [2]:
# Load the dataset
df = pd.read_csv('../data/raw/repositories_enriched.csv')

print(f"📊 Dataset loaded: {df.shape[0]:,} repositories × {df.shape[1]} columns")
print(f"🔤 Languages: {df['language'].nunique()}")
print(f"\n🔍 Sample columns:")
print(df.columns.tolist()[:20])

📊 Dataset loaded: 1,200 repositories × 52 columns
🔤 Languages: 12

🔍 Sample columns:
['id', 'name', 'full_name', 'owner', 'description', 'language', 'created_at', 'updated_at', 'pushed_at', 'stars', 'forks', 'watchers', 'open_issues', 'size_kb', 'license', 'has_wiki', 'has_pages', 'contributors_count', 'commits_30d', 'commits_90d']


## Engineered Features Function
This function adds all calculated fields using safe division to handle edge cases.

In [3]:
def add_engineered_features(frame: pd.DataFrame) -> pd.DataFrame:
    """
    Add calculated fields to repository dataframe.
    Uses NaN guards to prevent division by zero errors.
    """
    enriched = frame.copy()
    
    # Create safe denominators (replace 0 with NaN)
    contributors = enriched["contributors_count"].replace({0: np.nan})
    commits_365 = enriched["commits_365d"].replace({0: np.nan})
    size_kb = enriched["size_kb"].replace({0: np.nan})
    engagement_sum = enriched["stars"] + enriched["forks"] + enriched["watchers"]

    # Per-contributor metrics
    enriched["stars_per_contributor"] = enriched["stars"] / contributors
    enriched["forks_per_contributor"] = enriched["forks"] / contributors
    enriched["engagement_per_contributor"] = engagement_sum / contributors
    
    # Density metrics
    enriched["engagement_density"] = engagement_sum / size_kb
    
    # Momentum metrics
    enriched["recent_commit_share"] = enriched["commits_30d"] / commits_365
    enriched["quarter_commit_share"] = enriched["commits_90d"] / commits_365
    
    # Workload metrics
    enriched["issue_to_commit_ratio"] = enriched["open_issues"] / (enriched["commits_365d"] + 1)
    enriched["freshness_index"] = 1 / (1 + enriched["days_since_push"])
    enriched["support_load"] = enriched["open_issues"] / contributors
    
    # Governance metrics
    enriched["compliance_score"] = (
        enriched[["has_readme", "has_license", "has_contributing", "has_code_of_conduct"]]
        .astype(int)
        .mean(axis=1)
    )
    enriched["enterprise_ready"] = (
        enriched["has_license"]
        & enriched["has_contributing"]
        & enriched["has_code_of_conduct"]
    ).astype(bool)
    
    # Composite scores
    enriched["maturity_score"] = (
        0.4 * enriched["health_score"]
        + 0.35 * enriched["activity_score"]
        + 0.25 * enriched["popularity_score"]
    )
    enriched["growth_signal"] = (
        enriched["recent_commit_share"].fillna(0) * 0.5
        + enriched["quarter_commit_share"].fillna(0) * 0.3
        + (enriched["recency_score"].fillna(0) / 100) * 0.2
    )
    
    return enriched

print("✅ Feature engineering function defined")

✅ Feature engineering function defined


## Apply Engineered Features
Add all calculated fields to the dataset.

In [4]:
# Apply feature engineering
df_enriched = add_engineered_features(df)

print(f"✅ Features added!")
print(f"📊 New shape: {df_enriched.shape[0]:,} rows × {df_enriched.shape[1]} columns")
print(f"\n🆕 New calculated fields:")
new_cols = [col for col in df_enriched.columns if col not in df.columns]
for col in new_cols:
    print(f"   - {col}")

✅ Features added!
📊 New shape: 1,200 rows × 52 columns

🆕 New calculated fields:


## Add Segmentation Categories
Create categorical groupings for growth and compliance tiers.

In [5]:
# Growth segments
def categorize_growth(signal):
    if signal >= 0.7:
        return "High"
    elif signal >= 0.4:
        return "Medium"
    else:
        return "Low"

# Compliance tiers
def categorize_compliance(score):
    if score >= 0.75:
        return "Excellent"
    elif score >= 0.5:
        return "Good"
    elif score >= 0.25:
        return "Basic"
    else:
        return "Minimal"

df_enriched["growth_segment"] = df_enriched["growth_signal"].apply(categorize_growth)
df_enriched["compliance_tier"] = df_enriched["compliance_score"].apply(categorize_compliance)

print("✅ Segmentation categories added")
print(f"\n📊 Growth distribution:")
print(df_enriched["growth_segment"].value_counts())
print(f"\n📊 Compliance distribution:")
print(df_enriched["compliance_tier"].value_counts())

✅ Segmentation categories added

📊 Growth distribution:
growth_segment
Low       612
High      366
Medium    222
Name: count, dtype: int64

📊 Compliance distribution:
compliance_tier
Excellent    725
Good         426
Basic         49
Name: count, dtype: int64


## Export Enriched Dataset
Save the complete dataset with all engineered features.

In [6]:
# Export enriched dataset
output_path = "../data/derived/repositories_enriched.csv"
df_enriched.to_csv(output_path, index=False)

print(f"✅ Enriched dataset exported to: {output_path}")
print(f"📊 Total columns: {len(df_enriched.columns)}")

✅ Enriched dataset exported to: ../data/derived/repositories_enriched.csv
📊 Total columns: 52


---
## Derived Table 1: Language Summary
Aggregate key metrics by programming language for dashboard consumption.

In [7]:
# Language-level aggregations
language_summary = (
    df_enriched.groupby("language")
    .agg({
        "id": "count",
        "stars": "sum",
        "forks": "sum",
        "overall_score": "mean",
        "popularity_score": "mean",
        "activity_score": "mean",
        "health_score": "mean",
        "maturity_score": "mean",
        "growth_signal": "mean",
        "compliance_score": "mean",
        "support_load": "median",
        "contributors_count": "mean",
        "commits_365d": "mean",
    })
    .rename(columns={"id": "repo_count"})
    .round(2)
    .sort_values("overall_score", ascending=False)
 )

print("📊 Language Summary Table")
print("=" * 80)
print(language_summary)
print("\n✅ Language summary created")

📊 Language Summary Table
            repo_count    stars    forks  overall_score  popularity_score  \
language                                                                    
Rust               100  3461833   233757          49.40             27.07   
TypeScript         100  6860151   995677          48.33             16.06   
Go                 100  4301920   493763          46.92             22.98   
C++                100  3721389   683509          44.12             16.30   
Python             100  7810721  1315442          43.49             20.76   
JavaScript         100  5873437   897308          43.31             22.57   
Ruby               100  1464028   267438          42.40             21.40   
Java               100  3607982   981908          41.45             22.05   
Kotlin             100  1405327   218416          38.49             24.67   
PHP                100  1537067   282285          38.20             19.64   
Swift              100  1486132   154115          3

In [8]:
# Export language summary
language_summary_path = "../data/derived/language_comparison_summary.csv"
language_summary.to_csv(language_summary_path)
print(f"✅ Exported: {language_summary_path}")

✅ Exported: ../data/derived/language_comparison_summary.csv


---
## Derived Table 2: Top Growth Repositories
Identify repositories with the highest growth signals for spotlight analysis.

In [9]:
# Top growth repos
top_growth_repos = (
    df_enriched[["name", "full_name", "language", "stars", "forks",
                 "contributors_count", "growth_signal", "maturity_score",
                 "compliance_score"]]
    .sort_values(["growth_signal", "maturity_score"], ascending=False)
    .head(20)
    .reset_index(drop=True)
 )

print("🚀 Top 20 Growth Repositories")
print("=" * 80)
print(top_growth_repos[["name", "language", "growth_signal", "maturity_score"]].to_string(index=False))
print("\n✅ Top growth repos identified")

🚀 Top 20 Growth Repositories
          name   language  growth_signal  maturity_score
     PowerToys         C#            1.0       94.030947
    tensorflow        C++            1.0       90.863636
      SecLists        PHP            1.0       88.854839
  react-native        C++            1.0       85.239610
      mastodon       Ruby            1.0       83.807854
   firefox-ios      Swift            1.0       81.191480
      protobuf        C++            1.0       80.862793
           zed       Rust            1.0       80.739413
         servo       Rust            1.0       80.144119
      superset TypeScript            1.0       78.651276
          loki         Go            1.0       78.321151
           n8n TypeScript            1.0       77.810065
Signal-Android     Kotlin            1.0       77.561996
          nest TypeScript            1.0       77.082393
     coreutils       Rust            1.0       77.002800
        RSSHub TypeScript            1.0       76.643171
  

In [10]:
# Export top growth repos
top_growth_path = "../data/derived/top_growth_repos.csv"
top_growth_repos.to_csv(top_growth_path, index=False)
print(f"✅ Exported: {top_growth_path}")

✅ Exported: ../data/derived/top_growth_repos.csv


---
## Derived Table 3: Segment Summary
Cross-tabulation of growth segments and compliance tiers for portfolio analysis.

In [11]:
# Segment summary
segment_summary = pd.crosstab(
    df_enriched["growth_segment"],
    df_enriched["compliance_tier"],
    margins=True
 )

print("📊 Segment Summary (Growth × Compliance)")
print("=" * 80)
print(segment_summary)
print("\n✅ Segment summary created")

📊 Segment Summary (Growth × Compliance)
compliance_tier  Basic  Excellent  Good   All
growth_segment                               
High                 7        269    90   366
Low                 37        309   266   612
Medium               5        147    70   222
All                 49        725   426  1200

✅ Segment summary created


In [12]:
# Export segment summary
segment_summary_path = "../data/derived/segment_summary.csv"
segment_summary.to_csv(segment_summary_path)
print(f"✅ Exported: {segment_summary_path}")

✅ Exported: ../data/derived/segment_summary.csv


---
## Derived Table 4: Enterprise Readiness
Language-level enterprise readiness metrics for governance assessment.

In [13]:
# Enterprise readiness by language
enterprise_table = (
    df_enriched.groupby("language")
    .agg({
        "enterprise_ready": "sum",
        "id": "count",
        "support_load": "median",
        "compliance_score": "mean",
    })
    .rename(columns={
        "enterprise_ready": "enterprise_ready_count","id": "total_repos"
    })
    .round(2)
 )

enterprise_table["enterprise_ready_pct"] = (
    enterprise_table["enterprise_ready_count"] / enterprise_table["total_repos"] * 100
).round(1)

enterprise_table = enterprise_table.sort_values("enterprise_ready_pct", ascending=False)

print("🏢 Enterprise Readiness by Language")
print("=" * 80)
print(enterprise_table)
print("\n✅ Enterprise readiness table created")

🏢 Enterprise Readiness by Language
            enterprise_ready_count  total_repos  support_load  \
language                                                        
TypeScript                      40          100          1.61   
JavaScript                      35          100          1.14   
Rust                            33          100          1.94   
C++                             32          100          2.43   
Go                              32          100          1.41   
Ruby                            26          100          0.49   
C#                              25          100          2.67   
Python                          21          100          2.12   
Swift                           21          100          1.28   
Kotlin                          18          100          1.16   
Java                            15          100          2.04   
PHP                             12          100          0.45   

            compliance_score  enterprise_ready_pct  
l

In [14]:
# Export enterprise readiness table
enterprise_table_path = "../data/derived/enterprise_readiness_table.csv"
enterprise_table.to_csv(enterprise_table_path)
print(f"✅ Exported: {enterprise_table_path}")

✅ Exported: ../data/derived/enterprise_readiness_table.csv


---
## Summary and Validation

In [15]:
print("=" * 80)
print("🎉 ALL DERIVED TABLES GENERATED SUCCESSFULLY")
print("=" * 80)
print("\n📁 Exported Files:")
print("   1. ../data/derived/repositories_enriched.csv          - Full dataset with engineered features")
print("   2. ../data/derived/language_comparison_summary.csv    - Language-level aggregates")
print("   3. ../data/derived/top_growth_repos.csv              - Top 20 high-growth repositories")
print("   4. ../data/derived/segment_summary.csv               - Growth × Compliance matrix")
print("   5. ../data/derived/enterprise_readiness_table.csv    - Enterprise governance by language")
print("\n📊 Key Statistics:")
print(f"   • Total repositories: {len(df_enriched):,}")
print(f"   • Languages analyzed: {df_enriched['language'].nunique()}")
print(f"   • Engineered features: {len([col for col in df_enriched.columns if col not in df.columns])}")
print(f"   • High growth repos: {(df_enriched['growth_segment'] == 'High').sum():,}")
print(f"   • Excellent compliance: {(df_enriched['compliance_tier'] == 'Excellent').sum():,}")
print("\n✅ Ready for dashboard integration!")

🎉 ALL DERIVED TABLES GENERATED SUCCESSFULLY

📁 Exported Files:
   1. ../data/derived/repositories_enriched.csv          - Full dataset with engineered features
   2. ../data/derived/language_comparison_summary.csv    - Language-level aggregates
   3. ../data/derived/top_growth_repos.csv              - Top 20 high-growth repositories
   4. ../data/derived/segment_summary.csv               - Growth × Compliance matrix
   5. ../data/derived/enterprise_readiness_table.csv    - Enterprise governance by language

📊 Key Statistics:
   • Total repositories: 1,200
   • Languages analyzed: 12
   • Engineered features: 0
   • High growth repos: 366
   • Excellent compliance: 725

✅ Ready for dashboard integration!


---
## Next Steps

This notebook has successfully:
1. ✅ Applied engineered features to base dataset
2. ✅ Created segmentation categories
3. ✅ Generated language-level summaries
4. ✅ Identified top growth repositories
5. ✅ Produced enterprise readiness metrics

**Use these outputs in:**
- `06_calculation_documentation.ipynb` - Detailed metric documentation
- React Dashboard - Interactive visualizations
- Presentations - Summary statistics and insights